# Transport Case Study: Group 0 (Demo)

**Scenario**: Industrial Trichloroethylene (TCE) Spill

**Group Members**: 
- TODO: Add your names

**Date**: TODO

---

## 1. Overview and Learning Objectives

### Problem Statement
A 30-day trichloroethylene (TCE) spill from an industrial facility has contaminated the groundwater in the Limmat Valley. Your task is to:
1. Model the transport of TCE through the aquifer over a 10-year period
2. Analyze how the existing well field (pumping and injection wells) affects plume migration
3. Assess whether contamination reaches the Limmat River or compliance monitoring points
4. **(Optional for bonus credit)** Verify your numerical model against analytical solutions

### Learning Objectives
By completing this case study, you will:
- Set up a coupled MODFLOW-MT3DMS transport model
- Apply the telescope approach to refine resolution around source and wells
- Define contaminant source terms using the SSM package
- Analyze well-contaminant interactions (capture zones, spreading)
- **(Optional for bonus credit)** Verify numerical results with analytical solutions
- Communicate findings in a professional modeling report

### Deliverables
1. This completed notebook with all code executed and results displayed
2. Completed `case_config_transport.yaml` with justified parameter choices
3. Professional report (3-4 pages PDF) summarizing methods, results, and conclusions
4. **(Optional for bonus: +5-10%)** Analytical comparison section with plots and discussion

### Key Questions to Answer
- Will the pumping wells capture the TCE plume before it reaches the Limmat River?
- What is the maximum extent of the contamination (area where C > 5 mg/L)?
- When will contamination reach monitoring locations?
- How do injection wells/Sickergalerie affect plume spreading?
- **(Optional)** How well does a 1D analytical solution predict plume behavior compared to the full 2D/3D numerical model?

---
## 2. Workflow Summary

### Transport Case Study Workflow

This case study follows a simpler workflow than the flow case study (no scenario variations):

```
1. Load fresh base parent model (independent from your flow case results)
   ↓
2. Load well locations from flow case study (case_config.yaml)
   ↓
3. Define transport submodel domain (around wells and source area)
   ↓
4. Create refined grid for submodel (5m cells for better resolution)
   ↓
5. Extract boundary conditions from parent model
   ↓
6. Set up MODFLOW submodel with wells (steady-state flow)
   ↓
7. Run flow model and verify convergence
   ↓
8. Set up MT3DMS transport model (transient concentrations)
   ↓
9. Define contaminant source term (SSM package)
   ↓
10. Run 10-year transport simulation
    ↓
11. Post-process: concentration maps, breakthrough curves, mass balance
    ↓
12. Analyze well-contaminant interactions
    ↓
13. OPTIONAL (bonus): Analytical comparison (Ogata-Banks 1D)
    ↓
14. Interpret results and write professional report
```

### Key Concept: Steady Flow + Transient Transport

We assume **steady-state flow** (heads don't change with time) but **transient transport** (concentrations evolve over time). This is the standard approach for long-term contamination problems because:
- Groundwater flow reaches equilibrium quickly (days to weeks)
- Contaminant transport is much slower (months to years)
- Allows us to focus on transport processes without rerunning flow at each time step

### Differences from Flow Case Study

| Aspect | Flow Case Study | Transport Case Study |
|--------|----------------|---------------------|
| Starting model | Base parent model | Same fresh base parent model |
| Wells | Student implements from concession | **Reuse from case_config.yaml** |
| Scenarios | 3 stages + parameter variations | **Single run: wells + transport** |
| Complexity | 3-stage workflow | **Simpler: 1-stage** |
| Focus | Flow system response | **Contaminant fate and transport** |
| Analysis | Drawdown, river leakage | **Plume migration, breakthrough** |
| Time dimension | Steady-state | **Steady flow + transient transport** |
| Analytical check | Not applicable | **Optional (bonus credit)** |

### Time Estimate
- Setup and configuration: 2-3 hours
- Model execution and debugging: 2-3 hours
- Analysis and visualization: 2-3 hours
- **(Optional) Analytical comparison: 0.5-1 hour** → **+5-10% bonus**
- Report writing: 2-3 hours
- **Total: ~8-10 hours** (or 10-11 hours with bonus section)

---
## 3. Configuration and Setup

### Import Libraries

Import all necessary Python libraries for modeling, analysis, and visualization.

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import pickle
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from shapely.geometry import Point, Polygon
from shapely.affinity import rotate
import flopy
from flopy.discretization import StructuredGrid

# print current working directory
print("Current working directory: ", os.getcwd())

# Add the support repo to the path
sys.path.append(os.path.abspath('../../../SUPPORT_REPO/src'))
sys.path.append(os.path.abspath('../../../SUPPORT_REPO/src/scripts/scripts_exercises'))

# Import local modules
import case_utils 
from data_utils import download_named_file, get_default_data_folder
import grid_utils
from print_images import display_image
import plot_utils

### Load Configuration

Load transport scenario configuration from `case_config_transport.yaml`.

In [ ]:
# Load flow case study configuration
CASE_YAML = 'case_config.yaml'
cfg = case_utils.load_yaml(CASE_YAML)

# Get group configuration
group_number = cfg['group'].get('number', 0)
if not isinstance(group_number, int) or group_number < 0 or group_number > 8:
    raise ValueError("Group number must be an integer between 0 and 8.")

print(f"Group number: {group_number}")

# Load and merge the transport configuration
TRANSPORT_YAML = 'case_config_transport.yaml'
cfg_transport = case_utils.load_yaml(TRANSPORT_YAML)

# Merge transport config into cfg (transport-specific keys added to main config)
# This keeps both configs accessible from a single object
cfg.update(cfg_transport)

print(f"Configuration loaded successfully. Available sections: {list(cfg.keys())}")

---
## 4. Load Parent Flow Model

### Download and Load Base Model

Load the fresh base parent model (independent from your flow case study results). This ensures a known-good starting point for transport modeling.

**Important**: We use the baseline model, not your modified flow case study model, to:
- Avoid error propagation from flow modeling
- Start from a verified, converged flow field
- Simplify the workflow

In [ ]:
# Download parent base model and save it to the transport workspace
# To make sure not to carry over any unintended changes from the flow case study, 
# we download a fresh copy of the base model specified in the transport configuration.

# After cfg.update(cfg_transport), cfg['model'] now contains the transport model config
# with workspace pointing to the transport subdirectory
parent_base_model_name = cfg['model']['data_name']
parent_workspace = os.path.expanduser(cfg['model']['workspace'])

# Download to the transport-specific directory
parent_base_model_path = download_named_file(
    parent_base_model_name, 
    dest_folder=parent_workspace,
    data_type=None,  # Don't append additional subdirectory
)

# Handle zip file extraction if needed
if parent_base_model_path.endswith('.zip'):
    import zipfile
    extract_path = os.path.dirname(parent_base_model_path)
    with zipfile.ZipFile(parent_base_model_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    parent_base_model_path = os.path.join(extract_path, cfg['model']['namefile'])

print(f'Downloaded the parent base model to: {parent_base_model_path}')
print(f'Model workspace: {parent_workspace}')

### Verify Parent Model

Run the parent model to verify it converges and produces reasonable flow field.

In [ ]:
# ----- Load model results ----- #
parent_base_namefile = os.path.basename(parent_base_model_path)
m_parent_base = flopy.modflow.Modflow.load(
    parent_base_namefile, 
    model_ws=parent_workspace, 
    check=False, 
    forgive=False, 
    exe_name='mfnwt'
)
# Check if heads file exists, if not run the model
parent_hds_path = os.path.join(parent_workspace, f"{m_parent_base.name}.hds")
if not os.path.exists(parent_hds_path):
    print("Parent model heads file not found. Running parent model...")
    success, buff = m_parent_base.run_model(silent=True, report=True)
    if not success:
        raise RuntimeError("Parent model failed to run")
    print("✓ Parent model run completed")

# Load and visualize groundwater heads
headobj = flopy.utils.HeadFile(parent_hds_path)
print(f'Heads loaded from {parent_hds_path}')
heads = headobj.get_data()[0]  # Layer 0, stress period 0


# ----- Create visualization ----- #
# Create visualization
fig, ax = plt.subplots(figsize=(16, 12))

# Plot model with heads
pmv = flopy.plot.PlotMapView(model=m_parent_base, ax=ax)

# Plot head distribution as colored background
heads_masked = np.ma.masked_where(m_parent_base.bas6.ibound.array[0] <= 0, heads)
im = pmv.plot_array(heads_masked, alpha=0.6, cmap='Blues')

# Add head contours
contour_levels = np.linspace(np.nanmin(heads_masked), np.nanmax(heads_masked), 15)
cont = pmv.contour_array(heads_masked, levels=contour_levels, colors='black', 
                        linewidths=1.5, linestyles='-')
ax.clabel(cont, inline=True, fontsize=9, fmt='%.1f m')

# Plot model grid (light)
pmv.plot_grid(color='gray', alpha=0.3, linewidth=0.5)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.3, pad=0.02)
cbar.set_label('Hydraulic Head (m a.s.l.)', fontsize=12)

# Formatting
ax.set_title(f'Parent Base Model - Groundwater Flow Field\nGroup {group_number} - Hydraulic Heads', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('X Coordinate (m)', fontsize=12)
ax.set_ylabel('Y Coordinate (m)', fontsize=12)
ax.set_aspect('equal')

# Add text box with model info
info_text = f'Model: {m_parent_base.name}\nGrid: {m_parent_base.nrow}×{m_parent_base.ncol}\nCell size: {m_parent_base.dis.delr[0]:.0f}m'
ax.text(0.02, 0.98, info_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

---
## 5. Load Well Data from Flow Case Study

### Load Wells from case_config.yaml

Load the well locations and pumping/injection rates from your flow case study. Wells are **reused**, not reimplemented.

**Key Tasks:**
1. Read well data from flow case `case_config.yaml` or concession CSV
2. Identify which wells are **pumping** (negative Q) vs **injection** (positive Q)
3. Map well locations to parent model grid
4. Visualize well locations relative to planned source location

In [ ]:
# TODO: Load well data from case_config.yaml
# Parse well information
# Separate pumping vs injection wells
# Map to grid coordinates

# Load the rotated model grid for visualization
print(f"Constructiong modelgrid_path:")
modelgrid_path = os.path.join(os.path.dirname(parent_base_model_path), f"{parent_base_namefile.replace('.nam', '')}_modelgrid.pkl")
with open(modelgrid_path, 'rb') as f:
    parent_modelgrid = pickle.load(f)

print(f"Parent model grid loaded")
print(f"Grid rotation: {parent_modelgrid.angrot} degrees")
print(f"Grid extent: X [{parent_modelgrid.extent[0]:.1f}, {parent_modelgrid.extent[1]:.1f}]")
print(f"             Y [{parent_modelgrid.extent[2]:.1f}, {parent_modelgrid.extent[3]:.1f}]")

# Get well locations for the specified group
scenario = case_utils.get_scenario_for_group(CASE_YAML, group_number)
concession_id = scenario.get('concession', None)
if concession_id is None:
    raise ValueError(f"Concession ID not defined for group {group_number}")

# Load and filter wells by concession
well_data_path = download_named_file(name='wells', data_type='gis')
wells_gdf = gpd.read_file(well_data_path, layer='GS_GRUNDWASSERFASSUNGEN_OGD_P')
wells_gdf = case_utils.filter_wells_by_concession(wells_gdf, concession_id)

print(f"\nWells for concession {concession_id}:")
print(wells_gdf[['GWR_ID', 'GWR_PREFIX', 'FASSART']])

# Visualize wells on parent model
case_utils.plot_wells_on_model(m_parent_base, modelgrid=parent_modelgrid, wells_gdf=wells_gdf, 
                               concession_id=concession_id)

### Visualize Well Field

Plot well locations on model grid to understand spatial arrangement.

In [ ]:
# TODO: Create map showing:
# - Parent model grid
# - Pumping wells (red circles, size proportional to rate)
# - Injection wells (blue circles, size proportional to rate)
# - Planned source location (orange star)
# - Limmat River
# - Model boundaries

---
## 6. Define Transport Submodel Domain

### Estimate Plume Travel Distance

Before defining the submodel domain, estimate how far the plume might travel in 10 years:

**Travel distance** = velocity × time = (K·i/n) × t

Where:
- K = hydraulic conductivity (m/day) - from parent model
- i = hydraulic gradient (m/m) - from parent model heads
- n = effective porosity - from config (typically 0.25)
- t = simulation time (10 years = 3650 days)

**Rule of thumb**: Buffer should be at least 1.5× to 2× estimated travel distance to ensure plume stays within domain.

In [ ]:
# TODO: Extract K and gradient from parent model
# Calculate velocity v = K*i/n
# Estimate 10-year travel distance
# Suggest buffer distances for submodel

### Define Submodel Extent

Define the submodel domain boundaries. The domain should:
1. Include the contaminant source location
2. Include all wells from the well field
3. Have sufficient buffer for 10-year plume migration
4. Avoid placing boundaries where steep gradients are expected
5. Align with parent grid cells for easier boundary extraction

**Typical domain**: 1-2 km × 1-2 km around source and wells

In [ ]:
# TODO: Define submodel extent
# xmin, xmax, ymin, ymax (Swiss coordinates)
# Verify source and wells are inside domain
# Verify buffer is adequate
# Create polygon for submodel boundary

### Visualize Submodel Domain

Plot the submodel extent on the parent model grid.

In [ ]:
# TODO: Create map showing:
# - Parent model extent (light gray)
# - Submodel extent (bold rectangle)
# - Source location
# - Wells
# - Flow direction arrows
# - Limmat River

---
## 7. Create Telescope Submodel for Flow

### Why Telescope?

Transport modeling requires finer grid resolution than flow modeling because:
- Need to resolve source area (sharp concentration gradients)
- Peclet number constraint: Δx ≤ 2·αL (for αL=10m → Δx ≤ 20m)
- Parent model cells (~25-50m) are too coarse for accurate transport

The telescope approach:
1. Uses coarse parent model for regional flow (efficient)
2. Creates refined submodel only where needed (5m cells)
3. Extracts boundary conditions from parent model
4. Runs transport on refined grid (accurate)

### Create Refined Grid

Generate a refined grid for the submodel domain with 5m cell spacing.

In [ ]:
# TODO: Create refined grid
# Define cell spacing (5m)
# Calculate number of rows and columns
# Generate grid coordinates
# Verify grid quality (aspect ratio, alignment)

### Extract Boundary Conditions

Extract heads from parent model along submodel boundaries to use as boundary conditions.

In [ ]:
# TODO: Extract parent model heads at submodel boundaries
# Create CHD or GHB package for submodel
# Extract river cells within submodel (if any)
# Extract recharge (if applicable)

### Interpolate Aquifer Properties

Interpolate hydraulic conductivity, layer elevations, and other properties from parent to refined grid.

In [ ]:
# TODO: Interpolate from parent to submodel:
# - Hydraulic conductivity (K)
# - Layer top and bottom elevations
# - Specific storage (if transient)
# Visualize interpolated K field

### Set Up MODFLOW Submodel

Create MODFLOW submodel with refined grid, boundary conditions, and wells.

In [ ]:
# TODO: Create MODFLOW submodel
# - DIS package (refined grid)
# - BAS package (ibound, starting heads)
# - LPF/UPW package (K values)
# - CHD/GHB package (boundaries from parent)
# - WEL package (wells from flow case)
# - RIV package (if rivers in domain)
# - NWT solver

### Run Submodel Flow Simulation

Run steady-state flow on the refined submodel.

In [ ]:
# TODO: Write and run submodel
# Check convergence
# Load head file
# Load cell-by-cell file (for flow budget)

### Verify Submodel Flow Results

Check that submodel flow field is reasonable and consistent with parent model.

In [ ]:
# TODO: Quality checks
# - Mass balance error < 1%
# - Head contours match parent in interior
# - Flow directions reasonable
# - Wells create expected drawdown
# - No dry cells or convergence issues

# Visualize:
# - Head contour map
# - Drawdown from wells
# - Flow vectors

---
## 8. Set Up MT3DMS Transport Model

### Create MT3DMS Model Object

Link MT3DMS to the MODFLOW submodel for coupled transport simulation.

In [ ]:
# TODO: Create MT3DMS model
# mt = flopy.mt3d.Mt3dms(modelname='transport', modflowmodel=mf_sub)

### Basic Transport Package (BTN)

Set up the BTN package with porosity, initial conditions, and time stepping.

**Key parameters:**
- `prsity`: Effective porosity (0.25 for sand/gravel)
- `sconc`: Initial concentration (0 everywhere)
- `icbund`: Active transport cells (1=active, -1=constant, 0=inactive)
- `nprs`: Number of times to save concentration output

In [ ]:
# TODO: Create BTN package
# Define transport time steps and output times
# Set porosity from config
# Initialize concentration to zero

### Advection Package (ADV)

Configure the advection scheme. Recommended: TVD (Total Variation Diminishing) for accuracy and stability.

**Options for mixelm:**
- `mixelm=0`: MOC (Method of Characteristics) - accurate but slow
- `mixelm=1`: MMOC (Modified MOC) - faster, less accurate
- `mixelm=2`: HMOC (Hybrid MOC) - balance
- `mixelm=6`: TVD - **recommended** (accurate, stable, fast)

In [ ]:
# TODO: Create ADV package
# Use TVD scheme (mixelm=6)
# adv = flopy.mt3d.Mt3dAdv(mt, mixelm=6)

### Dispersion Package (DSP)

Set dispersivity values from configuration.

**Typical values:**
- Longitudinal dispersivity (αL): 10 m (scale-dependent, ~10% of travel distance)
- Transverse dispersivity (αT): 1 m (typically αL/10)
- Vertical dispersivity (αV): 0.1 m (typically αL/100)

**Molecular diffusion** is usually negligible compared to mechanical dispersion.

In [ ]:
# TODO: Create DSP package
# Load dispersivity values from config
# al = longitudinal dispersivity
# trpt = ratio of transverse to longitudinal (αT/αL)
# trpv = ratio of vertical to longitudinal (αV/αL)
# dsp = flopy.mt3d.Mt3dDsp(mt, al=10.0, trpt=0.1, trpv=0.01)

### Reaction Package (RCT) - If Applicable

For Group 0 (TCE), this is a **conservative tracer** (no sorption or decay), so RCT is not needed.

**For other groups with reactions:**
- **Sorption**: `isothm=1` (linear), `sp1=Kd` (distribution coefficient)
- **Decay**: `rc1=λ` (first-order decay constant, 1/day)

**Note**: Groups 3, 4, 7, 8 will need this package.

In [ ]:
# For Group 0: No RCT package needed (conservative tracer)

# For groups with reactions (example):
# rct = flopy.mt3d.Mt3dRct(mt, isothm=1, sp1=Kd, rc1=lambda_decay)

### GCG Solver Package

Configure the Generalized Conjugate Gradient solver for MT3DMS.

In [ ]:
# TODO: Create GCG package
# gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=100, iter1=50, cclose=1e-6)

---
## 9. Define Source Term (SSM Package)

### Map Source Location to Grid

Convert the source coordinates (Swiss LV03/LV95) to submodel grid indices (layer, row, column).

In [ ]:
# TODO: Load source location from config
# source_easting, source_northing from case_config_transport.yaml
# Convert to grid indices (layer, row, col)
# Verify source is within active model domain

### Visualize Source Location

Plot source location on submodel grid to verify correct placement.

In [ ]:
# TODO: Create map showing:
# - Submodel grid
# - Source cell (red star)
# - Wells
# - Head contours
# - Flow direction

### Set Up SSM Package

Define the source term using the Source-Sink Mixing (SSM) package.

**For Group 0 (30-day pulse):**
- Use `itype=-1` (constant concentration cell)
- Active for first 30 days (stress period 0)
- Zero concentration after 30 days (stress period 1+)

**SSM data format:**
```python
ssm_data = {
    0: [(lay, row, col, concentration, itype, 0, 0)],  # stress period 0
    1: [(lay, row, col, 0.0, itype, 0, 0)]             # stress period 1
}
```

In [ ]:
# TODO: Create SSM package
# Define stress periods (0-30 days: source active, 30-3650 days: source off)
# Set up SSM data dictionary
# ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=ssm_data)

---
## 10. Run Transport Simulation

### Write MT3DMS Input Files

Write all MT3DMS package files to disk.

In [ ]:
# TODO: Write MT3DMS input files
# mt.write_input()

### Run MT3DMS

Execute the transport simulation. This may take several minutes depending on grid size and time steps.

**Monitor for:**
- Convergence at each time step
- Mass balance errors
- Warnings about negative concentrations (instability)
- Courant/Peclet number violations

In [ ]:
# TODO: Run MT3DMS
# success, buff = mt.run_model(silent=False)
# Check for successful completion

### Load Concentration Results

Read the UCN (concentration) file to extract results at different times.

In [ ]:
# TODO: Load concentration results
# ucn = flopy.utils.UcnFile('MT3D001.UCN')
# Get times available
# Load concentrations at key times (30d, 365d, 1825d, 3650d)

---
## 11. Post-Processing and Visualization

### Concentration Maps at Multiple Times

Create concentration contour maps showing plume evolution over time.

**Suggested times:** 30 days, 1 year, 3 years, 5 years, 10 years

In [ ]:
# TODO: Create concentration maps
# For each time:
#   - Plot concentration contours
#   - Overlay wells, source, river
#   - Add scale bar, north arrow
#   - Label max concentration
# Create multi-panel figure showing evolution

### Breakthrough Curves at Monitoring Points

Plot concentration vs. time at key observation points.

In [ ]:
# TODO: Extract concentration time series at monitoring points
# Plot C(t) for:
#   - Downgradient monitoring well
#   - Pumping well location
#   - River compliance point
# Add compliance threshold line
# Calculate breakthrough time (when C > threshold)

### Plume Extent Over Time

Calculate the area where concentration exceeds a threshold (e.g., 1 mg/L or 5 mg/L).

In [ ]:
# TODO: For each time step:
# - Count cells where C > threshold
# - Calculate total area (n_cells × cell_area)
# - Plot extent vs time
# - Identify time of maximum extent

### Mass Balance Analysis

Track the fate of contaminant mass:
- Mass remaining in aquifer
- Mass exported through boundaries
- Mass captured by pumping wells
- Mass lost to decay (if applicable)

In [ ]:
# TODO: Calculate mass balance
# Initial mass = concentration × source volume × duration
# Mass in aquifer at each time = sum(C × cell_volume × porosity)
# Mass balance error from MT3DMS output
# Plot mass components vs time
# Verify mass balance error < 1%

---
## 12. Quality Checks

### Mass Balance Verification

Ensure mass is conserved throughout the simulation (error < 1%).

In [ ]:
# TODO: Extract mass balance from MT3DMS output
# Check cumulative mass balance error
# Flag if error > 1%

### Stability Criteria

Check Courant and Peclet numbers to verify numerical stability.

**Courant number (advective stability):**
```
Cr = v·Δt / Δx ≤ 1
```

**Peclet number (dispersive stability):**
```
Pe = v·Δx / D ≤ 2-4
```

Where:
- v = pore water velocity (m/day)
- Δt = time step (days)
- Δx = cell size (m)
- D = dispersion coefficient = αL·v

In [ ]:
# TODO: Calculate stability numbers
# Extract velocity from flow model
# Calculate Courant number
# Calculate Peclet number
# Verify both are within acceptable ranges

### Physical Reasonableness Checks

Verify results make physical sense.

In [ ]:
# TODO: Check for:
# - No negative concentrations
# - No concentrations > source concentration
# - Plume moves in expected flow direction
# - Maximum concentrations at reasonable locations
# - Plume shape consistent with dispersivity
# - Pumping wells show capture if within influence

---
## 13. Well-Contaminant Interaction Analysis

### Identify Pumping vs Injection Wells

Classify wells based on flow rates (negative = pumping, positive = injection).

In [ ]:
# TODO: Separate wells by type
# pumping_wells = wells with Q < 0
# injection_wells = wells with Q > 0
# Calculate total pumping and injection rates

### Pumping Well Capture Analysis

Assess whether pumping wells capture contamination.

**Key questions:**
- Do pumping wells intercept the plume?
- What concentration reaches the wells?
- What percentage of contaminant mass is captured?
- How does pumping affect plume migration?

In [ ]:
# TODO: For each pumping well:
# - Extract concentration time series at well location
# - Calculate time of breakthrough
# - Calculate peak concentration
# - Estimate mass captured
# Create capture zone map showing well influence on plume

### Injection Well Spreading Analysis

Assess whether injection wells spread contamination.

**Key questions:**
- Do injection wells push water toward or away from source?
- Could injection accelerate contaminant spreading?
- What is the zone of influence for injection wells?

In [ ]:
# TODO: For injection wells:
# - Map flow vectors showing injection influence
# - Compare plume extent with vs without injection (conceptual)
# - Identify if plume is pushed toward sensitive receptors

### Net Effect on Plume Migration

Synthesize the combined effect of all wells on contaminant transport.

In [ ]:
# TODO: Create summary visualization:
# - Concentration map with well capture zones
# - Flow vectors showing well influence
# - Plume centerline trajectory
# - Distance to compliance points

# Key findings table:
# - % mass captured by pumping wells
# - Maximum concentration at river
# - Time to reach compliance point
# - Protective vs. spreading effect of well field

---
## 14. OPTIONAL (Bonus Credit): Analytical Verification

### Why Consider Analytical Verification?

Analytical verification demonstrates professional modeling practice and deepens your understanding of when simple vs. complex methods are needed. **This section is optional but highly recommended** for:
- Earning **+5-10% bonus credit** on the assignment
- Understanding the limitations of 1D analytical solutions
- Learning when numerical modeling is truly necessary
- Demonstrating verification best practices

**If you choose to skip this section**, proceed directly to Section 15 (Sensitivity Analysis). Your report should focus on the numerical model results and well-contaminant interactions.

**If you choose to complete this section**, follow the Tier 1 requirements below.

---

### Tier 1 Requirements (Group 0 - Conservative Tracer)

Group 0 (conservative tracer) can perform full 1D Ogata-Banks comparison:
1. Extract 1D transect from MT3DMS results along flow direction
2. Implement 1D Ogata-Banks analytical solution
3. Plot comparison at multiple times
4. Calculate and plot breakthrough curves
5. Discuss discrepancies and their causes

**Time estimate**: 30-60 minutes with provided templates

---

### Extract 1D Transect from MT3DMS

Extract concentration along a transect from source in the direction of flow.

In [ ]:
# TODO: Define transect line
# - Start at source
# - Extend in flow direction (use velocity vector)
# - Sample concentrations along transect at multiple times
# - Store distance from source and concentration

### Implement Ogata-Banks Analytical Solution

Calculate 1D analytical solution with same parameters as MT3DMS.

**For continuous source:**
```python
C(x,t) = (C0/2) * [erfc((x-v*t)/(2*sqrt(D*t))) + 
                   exp(v*x/D) * erfc((x+v*t)/(2*sqrt(D*t)))]
```

**For pulse source (Group 0):**
Use superposition: solution at time t minus solution at time (t - t_pulse)

In [ ]:
# TODO: Implement Ogata-Banks function
# Input parameters from MT3DMS setup:
#   - v = pore water velocity (K*i/n)
#   - D = dispersion coefficient (alpha_L * v)
#   - C0 = source concentration
#   - t_pulse = 30 days

# Calculate analytical solution for same times as MT3DMS
# Handle pulse source with superposition

### Compare Concentration Profiles

Plot analytical vs numerical concentration along transect at multiple times.

In [ ]:
# TODO: Create comparison plots
# For times = [90d, 365d, 1825d, 3650d]:
#   - Plot C(x) analytical (solid line)
#   - Plot C(x) MT3DMS (points)
#   - Add legend, labels, title
# Create multi-panel figure showing evolution

### Compare Breakthrough Curves

Plot C(t) at a fixed location: analytical vs numerical.

In [ ]:
# TODO: Select observation distance (e.g., 200m downgradient)
# Calculate analytical C(t) at that distance
# Extract MT3DMS C(t) at nearest cell
# Plot both on same axes
# Calculate:
#   - Time of arrival (when C > 1% of C0)
#   - Peak concentration
#   - Time of peak

### Discuss Discrepancies

Analyze differences between analytical and numerical results.

**Expected differences due to:**
1. **2D/3D spreading**: Numerical model allows transverse dispersion, analytical is 1D only
2. **Grid discretization**: MT3DMS uses discrete cells, analytical is continuous
3. **Numerical dispersion**: Some artificial spreading from numerical scheme
4. **Well influence**: Wells affect flow field in 2D/3D but not in 1D analytical
5. **Boundary effects**: Numerical model has finite boundaries
6. **Heterogeneity**: Numerical model may have varying K, analytical assumes uniform

**Questions to address:**
- Where is agreement best? (near source vs far field, early vs late time)
- Are differences within acceptable uncertainty?
- When is analytical solution "good enough" vs when is numerical needed?
- What aspects cannot be captured by analytical methods?

In [ ]:
# TODO: Quantify differences
# Calculate RMSE between analytical and numerical
# Calculate relative error at peak concentration
# Identify times/locations with largest discrepancies
# Create table summarizing agreement metrics

### Conclusions from Analytical Comparison

Synthesize findings and answer: **When is analytical sufficient, and when is numerical modeling necessary?**

**If you completed this optional section, write 2-3 paragraphs discussing:**

1. **Quality of agreement**: How well did analytical match numerical? Where/when was agreement best?

2. **Causes of discrepancies**: Which factors (2D effects, wells, boundaries, etc.) caused the largest differences?

3. **When to use each method**: 
   - When would you recommend analytical solution for this problem?
   - What aspects required numerical modeling?
   - How would you approach a similar problem in the future?

**Bonus credit value**: This section can earn you +5-10% extra credit depending on depth of analysis and quality of discussion.

---
## 15. Sensitivity Analysis

### Parameter Sensitivity

Test how results change with ±50% variation in dispersivity.

**Rationale**: Dispersivity is scale-dependent and uncertain. Understanding sensitivity helps assess prediction reliability.

In [ ]:
# TODO: Run MT3DMS with:
# - Base case: αL = 10 m
# - Low case: αL = 5 m (less spreading)
# - High case: αL = 15 m (more spreading)

# Compare:
# - Plume extent at 10 years
# - Breakthrough time at monitoring point
# - Maximum concentration at compliance point

# Plot all three scenarios on same map

### Other Parameter Sensitivities (Optional)

If time permits, test sensitivity to:
- Porosity (affects velocity)
- Source concentration (linear scaling)
- Source duration (mass loading)
- Well pumping rates (capture efficiency)

---
## 16. Summary and Conclusions

### Key Findings

Summarize the main results of your transport modeling study.

**TODO: Summarize (bullet points):**

**Plume Behavior:**
- Maximum plume extent: X m² at Y years
- Maximum concentration at monitoring point: X mg/L at Y years
- Time to reach river/compliance point: X years (or "not reached in 10 years")

**Well Interactions:**
- Pumping wells captured X% of contaminant mass
- [Effect of injection wells on plume spreading]
- [Net protective or spreading effect of well field]

**Analytical Comparison (if completed):**
- Agreement quality: [good/moderate/poor] along flow transect
- Key discrepancies due to: [2D effects, wells, boundaries, etc.]
- Analytical would be sufficient for: [screening-level assessment]
- Numerical was needed for: [well interactions, accurate predictions]

**Sensitivity:**
- Results most sensitive to: [dispersivity, porosity, well rates]
- Uncertainty range on key predictions: ±X%

### Interpretation and Implications

Discuss what the results mean for the scenario.

**TODO: Write 2-3 paragraphs addressing:**

1. **Risk assessment**: Does contamination reach sensitive receptors? Are regulatory thresholds exceeded? What is the timeline for impact?

2. **Well field implications**: How do current pumping/injection operations affect contamination? Should well operations be modified?

3. **Uncertainty and limitations**: What are the main sources of uncertainty? What additional data would reduce uncertainty? What are model limitations?

4. **(If analytical comparison completed)** What insights did the analytical verification provide about when simple vs. complex models are needed?

### Recommendations

Provide actionable recommendations based on your findings.

**TODO: Recommend (bullet points):**

**Monitoring:**
- [Install monitoring wells at specific locations]
- [Sampling frequency and parameters]

**Well Operations:**
- [Adjust pumping rates if needed for capture]
- [Consider turning off/relocating injection if spreading contamination]

**Further Investigation:**
- [Additional site characterization needed]
- [Model refinements or alternative scenarios]

**Remediation (if applicable):**
- [Natural attenuation feasible? Time frame?]
- [Active remediation needed? What approach?]

---
## 17. Professional Report Preparation

### Report Structure (3-4 pages)

Your final deliverable is a professional PDF report with the following structure:

**1. Problem Statement and Objectives (0.5 page)**
- Brief description of TCE spill and well field
- Modeling objectives (key questions)
- Why numerical modeling was needed

**2. Methodology (0.75 page)**
- Transport model setup summary (domain, grid, parameters)
- Key assumptions and justification
- **(If completed)** Brief mention of analytical verification approach

**3. Results (1.5-2 pages, mostly figures)**
- **Figure 1**: Concentration map at 10 years with wells
- **Figure 2**: Breakthrough curve at critical location
- **Figure 3** (optional): Analytical vs numerical comparison (if completed)
- Brief text: Maximum concentrations, breakthrough times, plume extent
- Well-contaminant interaction summary

**4. Discussion and Conclusions (0.75-1 page)**
- Interpretation: What do results mean for scenario?
- **(If completed)** Analytical comparison: When is simple method sufficient?
- Parameter uncertainty: Which parameters matter most?
- Recommendations: Well management, monitoring, remediation

### Report Template

A Word/LaTeX template is provided in the course materials. Use it to ensure consistent formatting and professional appearance.

### Figure Quality Guidelines

- High resolution (at least 300 DPI)
- Clear labels and legends
- Descriptive captions
- Referenced in text
- Consistent color schemes

### Writing Tips

- **Be concise**: Every sentence should add value
- **Be specific**: "100 mg/L" not "high concentration"
- **Past tense**: "The model predicted..." not "The model predicts..."
- **Professional tone**: Technical but accessible
- **Cite sources**: FloPy, MT3DMS, parameter references

### Checklist Before Submission

- [ ] All figures have captions and are referenced in text
- [ ] Key results quantified (not just "the plume moved downgradient")
- [ ] **(If completed)** Analytical comparison included with discussion
- [ ] Recommendations are specific and actionable
- [ ] Report is 3-4 pages (not 10 pages!)
- [ ] Spell-checked and proofread
- [ ] PDF format (not Word doc)
- [ ] File named: `transport_report_group0.pdf`

### Grading Notes

**Base assignment (100%):**
- Technical implementation: 50%
- Professional report: 50%

**Bonus credit (optional):**
- Analytical verification (Section 14): +5-10% depending on quality and depth of analysis
- Total possible score: up to 110%

---
## Acknowledgments and References

### Software
- FloPy: Python package for MODFLOW (Bakker et al., 2016)
- MODFLOW-NWT: Groundwater flow model (Niswonger et al., 2011)
- MT3DMS: Multi-species transport model (Zheng and Wang, 1999)

### References

**TODO: Add references for:**
- TCE properties and parameter values
- Dispersivity scale relationships
- Ogata-Banks analytical solution
- Any other sources used

### Course Materials

This notebook is based on:
- Teaching notebook: `4b_transport_model_implementation.ipynb`
- Transport planning document: `transport_planning.md`
- Support repository: Analytical solution functions and plotting utilities

---

**End of Case Study**